<a href="https://colab.research.google.com/github/pokexpert/hw/blob/main/financeapihw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import json
import pandas as pd
import requests

#i got help from google on how to do this i searched up "how to format currency in python"
def format_currency(value):
    try:
        return f"${value:,.2f}"
    except:
        return value

In [57]:
def fetch_stock_data(stocks, apikey):
    qurl = "https://yfapi.net/v6/finance/quote"
    querystring = {"symbols": ",".join(stocks)}
    headers = {'x-api-key': apikey}

    response = requests.get(qurl, headers=headers, params=querystring)
    quote_data = response.json()

    stock_info_list = []
    if "quoteResponse" in quote_data and "result" in quote_data["quoteResponse"]:
        for stock in quote_data["quoteResponse"]["result"]:
            ticker = stock.get("symbol", "N/A")
            company = stock.get("shortName", "N/A")
            price = stock.get("regularMarketPrice", "N/A")
            stock_info_list.append({
                "Stock Ticker": ticker,
                "Company": company,
                "Current Market Price": format_currency(price) if price != "N/A" else "N/A"
            })
    return stock_info_list

In [58]:
def fetch_stock_metric(stocks, apikey, choice, options):
    metric_name, module, metric_key = options[choice]
    surl = "https://yfapi.net/v11/finance/quoteSummary/{}"
    sparams = {"modules": module}
    headers = {'x-api-key': apikey}

    stock_info_list = []
    for stock in stocks:
        response = requests.get(surl.format(stock), headers=headers, params=sparams)
        data = response.json().get("quoteSummary", {}).get("result", [{}])[0]
        metric_value = data.get(module, {}).get(metric_key, {}).get("raw", "N/A")
        stock_info_list.append({
            "Ticker Symbol": stock,
            metric_name: metric_value if metric_value != "N/A" else "N/A"
        })

    return stock_info_list

In [59]:
stocks = input("Enter stock symbols (comma separated): ").split(",")
print(f"You have entered: {', '.join(stocks)}")

apikey = "QS6GRszd8Y2Yd6RYTXdpY1czs9FNqr2U7gfReTIs"

stock_data = fetch_stock_data(stocks, apikey)
df = pd.DataFrame(stock_data)
print("\nStock Data:")
print(df)

Enter stock symbols (comma separated): AAPL,GOOGL
You have entered: AAPL, GOOGL

Stock Data:
  Stock Ticker        Company Current Market Price
0         AAPL     Apple Inc.              $238.03
1        GOOGL  Alphabet Inc.              $167.01


In [60]:
options = {
    "1": ("52 Week High", "summaryDetail", "fiftyTwoWeekHigh"),
    "2": ("52 Week Low", "summaryDetail", "fiftyTwoWeekLow"),
    "3": ("Return on Assets (ROA)", "financialData", "returnOnAssets")
}

choice = input("Select a metric:\n1. 52 Week High\n2. 52 Week Low\n3. Return on Assets (ROA)\nChoice: ")

if choice not in options:
    print("Invalid choice.")
else:
    metric_data = fetch_stock_metric(stocks, apikey, choice, options)
    df = pd.DataFrame(metric_data)
    print("\nStock Metrics Data:")
    print(df)

Select a metric:
1. 52 Week High
2. 52 Week Low
3. Return on Assets (ROA)
Choice: 1

Stock Metrics Data:
  Ticker Symbol  52 Week High
0          AAPL        260.10
1         GOOGL        207.05


In [64]:
trending_url = "https://yfapi.net/v1/finance/trending/US"
stock_details_url = "https://yfapi.net/v6/finance/quote"

try:
    trending_stocks = [stock['symbol'] for stock in requests.get(trending_url, headers={'x-api-key': apikey}).json()['finance']['result'][0]['quotes'][:5]]
    stock_data = requests.get(stock_details_url, headers={'x-api-key': apikey}, params={"symbols": ",".join(trending_stocks)}).json().get("quoteResponse", {}).get("result", [])

    df = pd.DataFrame([{
        "Symbol": s.get("symbol", "N/A"),
        "Name": s.get("shortName", "N/A"),
        "Current Price": format_currency(s.get("regularMarketPrice", "N/A")),
        "52 Week High": format_currency(s.get("fiftyTwoWeekHigh", "N/A")),
        "52 Week Low": format_currency(s.get("fiftyTwoWeekLow", "N/A"))
    } for s in stock_data])
    print(df)

except Exception as e:
    print(f"error: {e}")

  Symbol                             Name Current Price 52 Week High  \
0   NVDA               NVIDIA Corporation       $114.06      $153.13   
1    TSM  Taiwan Semiconductor Manufactur       $172.97      $226.40   
2    WBA   Walgreens Boots Alliance, Inc.        $10.26       $22.05   
3   TSLA                      Tesla, Inc.       $284.65      $488.54   
4   OKTA                       Okta, Inc.        $87.16      $114.50   

  52 Week Low  
0      $75.61  
1     $125.78  
2       $8.08  
3     $138.80  
4      $70.56  
